# Using Redis to store maintenance state

In [2]:
LAST_CHECKED = None
IS_UNDER_MAINTENANCE = False

def is_under_maintenance(conn):
    global LAST_CHECKED, IS_UNDER_MAINTENANCE
    if LAST_CHECKED < time.time() - 1:
        LAST_CHECKED = time.time()
        IS_UNDER_MAINTENANCE = bool(conn.get('is-under-maintenance'))
    return IS_UNDER_MAINTENANCE

# Storing configuration in Redis

In [3]:
def set_config(conn, type, component, config):
    conn.set(f'config:{type}:{component}', json.dumps(config))

In [4]:
CONFIGS = {}
CHECKED = {}

def get_config(conn, type, component, wait=1):
    key = f'config:{type}:{component}'
        
    # Check to see if we should update the configuration information
    # about the component.
    if CHECKED.get(key) < time.time() - wait:
        # We can, so update the time we checked this connection.
        CHECKED[key] = time.time()
        
        # Fetch the configuration for this component.
        config = json.loads(conn.get(key) or '{}')
        
        # Convert potentially Unicode keyword arguments into string keyword
        # arguments.
        config = dict((str(k), config[k]) for k in config)
        
        # Get the old configuration for this component.
        old_configs = CONFIGS.get(key)
        
        # Update the configuration if they are different.
        if config != old_config:
            CONFIGS[key] = config
    return CONFIGS.get(key)

# Automatic redis configuration management

In [5]:
REDIS_CONNECTIONS = {}

def redis_connection(component, wait=1):
    # Cache the key.
    key = f'config:redis:{component}'
    
    # Our wrapper takes a function and wraps it with another function.
    def wrapper(function):
        # Copy some useful metadata from the original function to the configuration handler.
        @functools.wrap(function)
        def call(*args, **kwargs):
            # Fetch the old configuration, if any.
            old_config = CONFIGS.get(key, object())
            # Get the new configuration, if any.
            _config = get_config(config_connection, 'redis', component, wait)
            
            config = {}
            
            # Make the configuration usable for creating a Redis connection.
            for k, v in _config.iteritems():
                config[k.encode('utf-8')] = v
            
            # If old and new configurations don't match, create a new connectin.
            if config != old_config:
                REDIS_CONNECTIONS[key] = redis.Redis(**config)
            return function(REDIS_CONNECTIONS.get(key), *args, **kwargs)

        # Return the fully wrapped function.
        return call

    # Return a function that can wrap our Redis function.
    return wrapper

In [ ]:
@redis_connection('logs')
def log_recent(conn, app, message):
    'the old log_recent() code'

log_recent('main', 'user 215 logged in')